## **Step 1- Preprocessing**

Step 1.1- Creating n grams

For analysing any sentence single single words as tokens is not enough. So, we need to treat some words together. For doing this we use a concept called- 'n-grams'. Here in our model we are going to use only bigrams. The input in this function much be in form of tokens.

In [57]:
def Create_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

Create_bigrams(['This', 'film', 'is', 'terrible'])

['This', 'film', 'is', 'terrible', 'This film', 'is terrible', 'film is']

Step 1.2 - Importing libraries

In [58]:
import torch
from torchtext import data
from torchtext import datasets
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import spacy
from torchtext import vocab

Step 1.3 - Loading data and do splitting

In [59]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = 'spacy', 
                  preprocessing = Create_bigrams, 
                  #tokenizer_language='en_core_web_sm',
                  lower=True)
LABEL = data.LabelField(dtype = torch.float)

In [60]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Step 1.4 - Build the vocab

In [69]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

Step 1.5 - create the number of iterators

In [70]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

# **Step 2 - Create model**

Step 2.1 - calculating the word embedding 

In [71]:
class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):

        embedded = self.embedding(text)
        embedded = embedded.permute(1, 0, 2)
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        return self.fc(pooled)

In [72]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

Step 2.2 - Counting Parameters

In [73]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [74]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.0715,  0.9198,  1.4629,  ..., -0.4928, -0.1334,  1.6020],
        [-0.3658,  0.4468,  1.6586,  ..., -0.6352, -2.0189, -0.6479],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 1.9672, -0.0860, -0.1657,  ...,  0.1853, -1.3095,  0.1087],
        [ 1.7140,  0.7705, -0.9322,  ..., -0.7253,  0.4195,  0.9191],
        [-0.2280,  0.0721,  0.6030,  ..., -0.2734, -0.0248,  0.9652]])

In [75]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# **Step 3 - Train the model**

Step 3.1 - Initializing our optimizer

In [76]:
optimizer = optim.Adam(model.parameters())

Step 3.2 - Defining criteria

In [77]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

Step 3.3 - Function for calculating accuracy

In [79]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

Step 3.4 - Model Training

In [80]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Step 3.5 - Evaluation function for our model and calculating time.


In [85]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [86]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Step 3.6 - Printing epochs details

In [87]:

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model_weights.h5')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 8s
	Train Loss: 0.664 | Train Acc: 71.18%
	 Val. Loss: 0.536 |  Val. Acc: 75.09%
Epoch: 02 | Epoch Time: 1m 8s
	Train Loss: 0.595 | Train Acc: 78.41%
	 Val. Loss: 0.442 |  Val. Acc: 79.70%
Epoch: 03 | Epoch Time: 1m 8s
	Train Loss: 0.516 | Train Acc: 83.40%
	 Val. Loss: 0.394 |  Val. Acc: 83.14%
Epoch: 04 | Epoch Time: 1m 7s
	Train Loss: 0.442 | Train Acc: 86.79%
	 Val. Loss: 0.378 |  Val. Acc: 85.49%
Epoch: 05 | Epoch Time: 1m 8s
	Train Loss: 0.389 | Train Acc: 88.53%
	 Val. Loss: 0.382 |  Val. Acc: 86.63%


Step 3.7 - Calculating final accuracy after training

In [89]:
model.load_state_dict(torch.load('model_weights.h5'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.380 | Test Acc: 85.13%


# **Step 4 - Checking model by providing user input**

In [97]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = Create_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    if prediction.item() >= 0.5:
      return print('positive review')
    else:
      return print('negative review')

In [98]:
predict_sentiment(model, "What a lovely piece of shit this was")

negative review


In [99]:
predict_sentiment(model, "What a lovely piece this was")

positive review


In [100]:
predict_sentiment(model, "very nice")

positive review
